# Часть 1: ограничения и защита веб-сайтов

[Небольшой ликбез](https://developer.mozilla.org/ru/docs/Learn_web_development/Howto/Web_mechanics/What_is_a_domain_name) по структуре доменных имён (посмотрите после семинара)

## Файл robots.txt 

Файл `robots.txt` — это специальный текстовый файл, который веб-сайты размещают в корневом каталоге (если адрес сайта -`example.com`, то путь до этого файла будет`example.com/robots.txt`). Он содержит инструкции для веб-краулеров (ботов), определяя, какие страницы можно или нельзя сканировать.

💡 Основная цель robots.txt:

- Контролировать поведение поисковых ботов (Googlebot, Bingbot и др.).
- Ограничивать доступ к конфиденциальным или незначимым страницам.
- Предотвращать перегрузку сервера большим количеством запросов.

Пример содержимого robots.txt:

```
User-agent: *
Disallow: /admin/*
Disallow: /login/*
Allow: /public/*
Sitemap: https://example.com/sitemap.xml
```

Что к чему:

- User-agent: * — правило применимо ко всем ботам (если стоит название конкретного бота - значит к конкретному)
- Disallow: /admin/* — запрещает доступ к /admin/ и его подпапкам
- Allow: /public/ — разрешает доступ к /public/ и его подпапкам
- Sitemap: указывает на `sitemap.xml`, который содержит список страниц.

Можете посмотреть на пример robots.txt для hh.ru: [**ссылка**](https://hh.ru/robots.txt)


__Что здесь важного для нас?__

- `robots.txt` не является чем-то, что юридически обязательно размещать, но соблюдение его условий является этичным
- Некоторые сайты могут использовать его для предотвращения скрэппинга --> т.е. наказывать за несоблюдение
- Запрет в robots.txt не означает, что страницу нельзя загрузить вручную через браузер — он просто даёт указание ботам
- Если указать `User-agent` в заголовке запроса, такой запрос будет симулировать запрос браузера (а не поисковых роботов), поэтому возможные ограничения (блок по IP или правовые проблемы [если речь идет прямо о серьезных масштабах] __скорее всего__ не последуют)

__Мораль:__ прикрепляйте  `User-agent` в заголовок запроса при скрэппинге :) 

Некоторые сайты без этого поля в заголовке запроса не отправят вам запрашиваемую веб-страницу вовсе. Про `robots.txt` подробнее можно почитать [здесь](https://brightdata.com/blog/how-tos/robots-txt-for-web-scraping-guide)

---

### Что такое `sitemap.xml`

Файл `sitemap.xml` (может называться немного по-другому; как правило он указывается в `robots.txt`, так что можно сделать `ctrl+f xml` и скорее всего найти его) содержит список URL, которые сайт рекомендует поисковым системам для индексации.

📌 Важно:

- В `sitemap.xml` могут быть не все страницы сайта. Т.е. это не полная структура сайта, однако та, которая рекомендуется для скрэппинга поисковым системам
- Наличие страницы в `sitemap.xml` не гарантирует, что она появится в поиске.
- Поисковые системы могут индексировать и страницы, которых нет в `sitemap.xml`, если они найдены, например, по ссылкам на сайте

__Что здесь важного для нас, людей, которые скрэпят сайт?__


- `sitemap.xml` позволяет получить список \*почти всех\* доступных страниц (полезно, если мы хотим либо получить полную картину сайта перед какой-то основной задачей, либо в целом автоматизировать его парсинг [что нужно делать в целом аккуратно])
- Поскольку адреса всех страниц уже есть на руках, мы немного освобождаемся от лимитов на частоту запросов (может быть полезно, но не всегда)
- Некоторые сайты в `sitemap.xml` добавляют метаданные, которые можно использовать для оптимизации парсинга.

Пример:

```
<url>
    <loc>https://example.com/article1</loc>
    <lastmod>2024-02-10</lastmod>
    <changefreq>daily</changefreq>
    <priority>0.8</priority>
</url>
```
- lastmod --> когда страница последний раз обновлялась. Можно парсить только свежие страницы.
- changefreq --> как часто обновляется контент (например, daily, monthly). Можно оптимизировать частоту запросов.
- priority --> приоритетность страницы (например, 1.0 — очень важная, 0.1 — малозначимая).

# Часть 2: достаём данные со статического сайта

### Теория: HTML

Веб-страница — это документ, написанный на HTML (HyperText Markup Language)

Она содержит структурированные элементы, которые браузер интерпретирует и отображает пользователю. Вот пример того, что мы можем видеть при открытии html-кода веб-страницы:


```
<!DOCTYPE html>
<html>
<head>
    <title>Пример страницы</title>
</head>
<body>
    <h1>Заголовок страницы</h1>
    <p class="description">Это абзац текста.</p>
    <a href="https://example.com">Ссылка на сайт</a>
    <ul>
        <li class="item">Элемент 1</li>
        <li class="item">Элемент 2</li>
    </ul>
</body>
</html>
```

- `<!DOCTYPE html>` — объявляет HTML5-документ.
- `<html>` — корневой элемент страницы.
- `<head>` — содержит метаинформацию и подключение стилей/скриптов.
- `<body>` — содержит контент, который видит пользователь.

---
__Выглядит вот так в браузере:__



<!DOCTYPE html>
<html>
<head>
    <title>Пример страницы</title>
</head>
<body>
    <h1>Заголовок страницы</h1>
    <p class="description">Это абзац текста.</p>
    <a href="https://example.com">Ссылка на сайт</a>
    <ul>
        <li class="item">Элемент 1</li>
        <li class="item">Элемент 2</li>
    </ul>
</body>
</html>


**Основные HTML-теги:**

| Тег | Назначение |
| --- | --- |
| `<h1> - <h6>` | Заголовки разного уровня |
| `<p>` | Абзац текста |
| `<a>` | Ссылка (`href="..."`) |
| `<ul>` / `<ol>` | Список (неупорядоченный / упорядоченный) |
| `<li>` | Элемент списка |
| `<div>` | Блоковый контейнер |
| `<span>` | Строчный контейнер |
| `<img>` | Изображение (`src="..."`) |

⚡ **Важные атрибуты:**

*   `class="..."` — определяет CSS-класс элемента
*   `id="..."` — уникальный идентификатор элемента
*   `href="..."` — ссылка в `<a>`
*   `src="..."` — путь к изображению в `<img>`

### Теория: Beautifulsoup

In [3]:
from bs4 import BeautifulSoup

html_doc = """
<html>
    <head><title>Пример страницы</title></head>
    <body>
        <h1>Заголовок</h1>
        <p class="info">Пример абзаца с <a href="https://example.com">ссылкой</a>.</p>
        <p class="info 2">Пример абзаца с <a href="https://example.com">ссылкой 2</a>.</p>
    </body>
</html>
"""

soup = BeautifulSoup(html_doc, "lxml")  # html.parser

print(soup.prettify())  # Выводит HTML с красивым форматированием

<html>
 <head>
  <title>
   Пример страницы
  </title>
 </head>
 <body>
  <h1>
   Заголовок
  </h1>
  <p class="info">
   Пример абзаца с
   <a href="https://example.com">
    ссылкой
   </a>
   .
  </p>
  <p class="info 2">
   Пример абзаца с
   <a href="https://example.com">
    ссылкой 2
   </a>
   .
  </p>
 </body>
</html>



Основной принцип работы с BeautiflSoup состоит в следующем:
- Найти нужный нам элемент (например, через веб-интерфейс) и понять, где в структуре html-страницы он лежит
- Добраться до него (через `find` или `find_all`)
- Извлечь нужную нам информацию (например, через html-атрибуты ИЛИ через теги)

__Основные методы поиска элементов__


- `.find("tag")`	Находит первый элемент с указанным тегом
- `.find_all("tag")`	Находит все элементы с указанным тегом
- `.select("CSS селектор")`	Находит элементы по CSS-селектору


__Примеры:__

In [4]:
# Найти первый <h1>
h1 = soup.find("h1")
print(h1.text)  # ➝ "Заголовок"

# Найти все <p>
all_p = soup.find_all("p")
print(all_p)  # ➝ [<p class="info">...</p>]

# Найти элемент по CSS-селектору
css_select = soup.select(".info")
print(css_select)  # ➝ [<p class="info">...</p>]

Заголовок
[<p class="info">Пример абзаца с <a href="https://example.com">ссылкой</a>.</p>, <p class="info 2">Пример абзаца с <a href="https://example.com">ссылкой 2</a>.</p>]
[<p class="info">Пример абзаца с <a href="https://example.com">ссылкой</a>.</p>, <p class="info 2">Пример абзаца с <a href="https://example.com">ссылкой 2</a>.</p>]


In [8]:
# Достать отдельно тег <a> из первого тега <p>
print(all_p[0].find("a"))
# Достать текст тег <a> из первого тега <p>
print(all_p[0].find("a").text)

<a href="https://example.com">ссылкой</a>
ссылкой


Посмотрим на то, как достать только содержимое \<p> тега (то есть текст "ссылкой" из тега <a> внутри мы уберем).

Из очевидного решения приходят всяческие замены (str.replace() [или re.sub(), если вы уже с этим знакомы]), но мы посмотрим на то, как это делать инструментами bs4

In [31]:
first_p_tag = all_p[0]  # достаем первый тег

# если мы попытаемся достать текст без дополнительных параметров, нам будет выдаваться текст всех тегов, содержащихся внутри <p>
print(first_p_tag.get_text()) 

Пример абзаца с ссылкой.


давайте посмотрим, как вообще устроен find_all (и find)

In [32]:
help(first_p_tag.find_all)

Help on method find_all in module bs4.element:

find_all(name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs) method of bs4.element.Tag instance
    Look in the children of this PageElement and find all
    PageElements that match the given criteria.
    
    All find_* methods take a common set of arguments. See the online
    documentation for detailed explanations.
    
    :param name: A filter on tag name.
    :param attrs: A dictionary of filters on attribute values.
    :param recursive: If this is True, find_all() will perform a
        recursive search of this PageElement's children. Otherwise,
        only the direct children will be considered.
    :param limit: Stop looking after finding this many results.
    :kwargs: A dictionary of filters on attribute values.
    :return: A ResultSet of PageElements.
    :rtype: bs4.element.ResultSet



Иными словами, нам вообще-то необязательно указывать название тегов, которые мы ищем, и вот как раз пример, когда мы можем искать только строки

In [34]:
# Нам будет возвращен список всех строк (обратите внимание, строка от тега <a> посередине)
print(first_p_tag.find_all(string=True))

# У метода `find_all` есть параметр `recursive`, с помощью которого мы можем отключить поиск с вложенностью:
print(first_p_tag.find_all(string=True, recursive=False))

['Пример абзаца с ', 'ссылкой', '.']
['Пример абзаца с ', '.']


Выбросив ненужный вложенный тег, осталось только объединить результат в одну строку!

In [37]:
# В данном случае .strip() не даёт ничего [т.к. пробелов по бокам нет]
# но использовать обрезку при парсинге часто помогает, поэтому оставим как good practice
text = ''.join(first_p_tag.find_all(string=True, recursive=False)).strip()
print(text)  

Пример абзаца с .


 __Извлечение текста и атрибутов__
- `.text` извлекает текст из элемента
- `["атрибут"]`	- достает значение атрибута

**Примечание** атрибут `.text` извлечет вообще весь текст (на примере выше увидели, что это будет затрагивать вложенные теги)

__Примеры:__

In [40]:
html_doc = """
<html>
    <head><title>Пример страницы</title></head>
    <body>
        <h1>Заголовок</h1>
        <p class="info">Пример абзаца с <a href="https://example.com">ссылкой</a>.</p>
    </body>
</html>
"""

soup = BeautifulSoup(html_doc, "lxml")
# ----------

p = soup.find("p")

# Извлечение текста
print(p.text)  # ➝ "Пример абзаца с ссылкой."

# Извлечение атрибута href
a = p.find("a")
print(a["href"])  # ➝ "https://example.com"


Пример абзаца с ссылкой.
https://example.com


Про модуль csv:

https://pythonworld.ru/moduli/modul-csv.html

### Практика

Мы будем доставать данные с яндекс-маркета

__Вам необходимо:__
- Придумать товар, который вы захотите достать из маркета
- Собрать информацию по пяти страницам выдачи, которая содержит следующую информацию:
    - Цена
    - Бренд
    - Название товара
    - Рейтинг
    - Число оценок
- Сохранить информацию в .csv файл

Из полезного здесь:

- find_all("...", recursive=False) -> чтобы находить только high-level тэги

**Примечание** Данный ноутбук является уже заполненным для поиска конкретного товара (ложного септума)

### Пошаговый пример

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

URL = "https://market.yandex.ru/search"
params = {"text": "септум ложный", 
         "page": "1"}
headers = {"User-Agent": "",  # User-Agent вставьте из браузера (используя инструменты разработчика)
           "Cookie": "".encode("utf-8")}  # Cookie вставьте из браузера (используя инструменты разработчика)

resp = requests.get(URL, params=params, headers=headers)

In [ ]:
soup = BeautifulSoup(resp.text, "lxml")

In [ ]:
all_items = soup.find_all("div", class_="_1H-VK")
all_items = all_items[11:]  # убираем первые 11 продуктов из промо-выдачи


sample_item = all_items[-1]
span_container = sample_item.find("div", class_="_3iCDs aP0JE").find("span", class_="ds-valueLine ds-valueLine_gap_2")
product_price = span_container.find("span").text


info_container = sample_item.find("div", class_="_1ENFO")

brand_name = info_container.find("a").find("span")
if brand_name is not None:
    brand_name = brand_name.text

product_name = info_container.find("a").find("p")
if product_name is not None:
    product_name = product_name.text

rating = info_container.find("span", "ds-text ds-text_weight_bold ds-text_color_text-rating ds-text_proportional ds-text_typography_text Ys3yn ds-text_text_tight ds-text_text_bold").text
n_rates = info_container.find("span", "ds-text ds-text_lineClamp_1 ds-text_weight_reg ds-text_color_text-secondary ds-text_proportional ds-text_typography_text ds-text_text_tight ds-text_text_reg ds-text_lineClamp").text.strip()

product_price, brand_name, product_name, rating, n_rates

### Полный пример (объединяем всё вместе)

In [4]:
items_found = []
URL = "https://market.yandex.ru/search"
headers = {"User-Agent": "",  # User-Agent вставьте из браузера (используя инструменты разработчика)
           "Cookie": "".encode("utf-8")}  # Cookie вставьте из браузера (используя инструменты разработчика)


for i in tqdm(range(5)):
    params = {"text": "септум ложный", 
         "page": f"{i+1}"}
    resp = requests.get(URL, params=params, headers=headers)

    soup = BeautifulSoup(resp.text, "lxml")

    all_items = soup.find_all("div", class_="_1H-VK")
    if i == 0:
        all_items = all_items[11:]  # 11: , чтобы избавиться от первых 11 промо-айтемов



    for sample_item in all_items:
        span_container = sample_item.find("div", class_="_3iCDs aP0JE").find("span", class_="ds-valueLine ds-valueLine_gap_2")
        product_price = span_container.find("span").text
        
        
        info_container = sample_item.find("div", class_="_1ENFO")
        
        brand_name = info_container.find("a").find("span")
        if brand_name is not None:
            brand_name = brand_name.text
        
        product_name = info_container.find("a").find("p")
        if product_name is not None:
            product_name = product_name.text
        
        rating = info_container.find("span", "ds-text ds-text_weight_bold ds-text_color_text-rating ds-text_proportional ds-text_typography_text Ys3yn ds-text_text_tight ds-text_text_bold")
        if rating is not None:
            rating = rating.text
        
        n_rates = info_container.find("span", "ds-text ds-text_lineClamp_1 ds-text_weight_reg ds-text_color_text-secondary ds-text_proportional ds-text_typography_text ds-text_text_tight ds-text_text_reg ds-text_lineClamp")
        if n_rates is not None:
            n_rates = n_rates.text
        
        
        items_found.append((product_price, brand_name, product_name, rating, n_rates))
        

100%|█████████████████████████████████████████████| 5/5 [00:07<00:00,  1.57s/it]


Сохраняем в csv

In [ ]:
import csv
with open('data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["price", "brand_name", "name", "rating", "n_ratings"])
    writer.writerows(items_found)

# Практика на дом

Попробуйте сами воспроизвести пример выше:
- Выберите любой товар на яндекс-маркете (с которым мы ещё не работали) и достаньте первые 5 страниц выдачи (достаньте первые 11 промо-продуктов в выдаче тоже)
- Попробуйте также достать:
    -  цену в сплит
    -  ссылку на изображение товара на странице выдачи (thumbnail)
    -  ссылку на саму страницу товара (ту, переходя по которой мы получаем всю подробную информацию о товаре, отзывы и т.д.)
- Сохраните результат в csv


Если захочется чего-то ещё, попробуйте повыставлять на сайте фильтры (которые присутствуют слева - типа бренда/стоимости/т.д.) и посмотреть через инструменты разработчика, как от этого меняется запрос - попробуйте воспроизвести такие запросы через requests 

# Ридинги + полезные материалы
Некоторые ссылки повторяются из слайдов лекций, но я всё равно включил, чтобы собрать в одном месте

__Про HTTP-запросы:__
- [HTTP-запросы: структура, методы, строка статуса и коды состояния](https://habr.com/ru/articles/865040/)
- [Видео про устройство протокола HTTP](https://www.youtube.com/watch?v=RlccXUx4LVw)

__Про requests__:
- [Python requests library guide](https://realpython.com/python-requests/)
- [раздел quickstart в доке requests - можно глянуть ради интереса](https://requests.readthedocs.io/en/latest/user/quickstart/)
  
__Про bs4__:
- [Beautiful Soup: Build a Web Scraper With Python](https://realpython.com/beautiful-soup-web-scraper-python/)
- [Дока Beautiful Soup (просто почитать до раздела `Kinds of objects`)](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

__Бонус__:
- [Прикольный видос, связанный с темой доступа к открытым данных (и что вообще из них можно получить)](https://youtu.be/frLydE1UCvA)
- [Видос про парсинг стима ](https://youtu.be/qiNv3qv-YbU?si=JJ7czJJ5vza757Vl) (ссылок на скрэппер там нет, но вы можете поискать что-то похожее на гитхабе по запросу `steam scrapping` и выставить слева фильтр `Python`)